![bse_logo_textminingcourse](https://bse.eu/sites/default/files/bse_logo_small.png)

# Big Data Management - Assignment 1
## Document Stores

### by Luis Francisco Alvarez Poli, Mikel Gallo, Clarice Mottet

0. **[Part 0: Set Up](#part0)**
- **Objective**: Initialize programming environment.

1. **[Part 1: Model Creation](#part1)**
- **Objective**: Create three modeling alternatives using MongoDB.
- **Tasks:**
  - Model1: Two types of documents, one for each class and referenced fields.
  - Model2: One document for “Person” with “Company” as embedded document.
  - Model3: One document for “Company” with “Person” as embedded documents.

2. **[Part 2: Query Execution](#part2)**
- **Objective**: Execute four queries and log run time for each model.
- **Tasks:**
  - Query1: For each person, retrieve their full name and their company’s name.
  - Query2: For each company, retrieve its name and the number of employees.
  - Query3: For each person born before 1988, update their age to “30”.
  - Query4: For each company, update its name to include the word “Company”.

3. **[Part 3: Results & Discussion](#part3)**
- **Objective**: Compare run times for query execution across the three models.
- **Tasks:** 
  - Question1: Order queries from best to worst for Q1. Which model performs best? Why?
  - Question2: Order queries from best to worst for Q2. Which model performs best? Why?
  - Question3: Order queries from best to worst for Q3. Which model performs best? Why?
  - Question4: Order queries from best to worst for Q4. Which model performs best? Why?
  - Question5: What are your conclusions about denormalization or normalization of data in MongoDB? In the case of updates, which others better performance?


## <a id='part0'>Part 0: Set Up</a>
- **Objective**: Initialize programming environment.

In [10]:
#libraries
import datetime
import time
import json
from pymongo import MongoClient
from bson.objectid import ObjectId
from faker import Faker
import pandas as pd
import numpy as np
import random

#global variables
NUMBER_OF_COMPANIES = 2
NUMBER_OF_EMPLOYEES = 4

## <a id='part1'>Part 1: Model Creation</a>
- **Objective**: Create three modeling alternatives using MongoDB.
- **Tasks:**
  - Model1: Two types of documents, one for each class and referenced fields.
  - Model2: One document for “Person” with “Company” as embedded document.
  - Model3: One document for “Company” with “Person” as embedded documents.

### **Model1**: Two types of documents, one for each class and referenced fields.

In [2]:
#Model1

class Model1:
    #initialize the collection to hold two types of documents
    def __init__(self, host='127.0.0.1', port=27017, dbname='test'):
        self.client = MongoClient(host, port)
        self.db = self.client[dbname]
        self.db.drop_collection("model1")
        self.collection = self.db.create_collection('model1')

    #create a function that generates data and stores it in the collection
    def data_generator(self, n_company, n_person):
        #create sample data
        fake = Faker(['en_US'])

        #create sample company data
        for c in range(n_company):
            company = {
                'type': 'company',
                'domain': fake.domain_word(),
                'email': fake.ascii_company_email(),
                'name': fake.company(),
                'url': fake.uri(),
                'vatNumber': fake.nic_handles(),
                'staff': []
            }
            company_id = self.collection.insert_one(company).inserted_id
            list_persons = []
            
            #create sample staff data
            for p in range(n_person):
                today = datetime.date.today()
                dob = pd.to_datetime(fake.date_of_birth(minimum_age = 18, maximum_age = 67))
                age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
                person = {
                    'type': 'person',
                    'age': age,
                    'companyEmail': fake.ascii_company_email(),
                    'dateOfBirth': dob,
                    'email': fake.email(),
                    'firstName': fake.first_name(),
                    'secondName': fake.last_name(),
                    'job': fake.job(),
                    'worksIn': company_id
                }
                list_persons.append(person)
            
            inserted_persons = self.collection.insert_many(list_persons)
            self.collection.update_one(
                {'_id': company_id},
                {'$set': {'staff': inserted_persons.inserted_ids}}
            )

    #query1: prints full name and company for all persons
    def query1(self):
        start_time = time.time()
        for person in self.collection.find({"type": "person"}):
            full_name = person['firstName'] + ' ' + person['secondName']
            company = self.collection.find_one({'_id':person['worksIn']})
            company_name = company['name']
            print(" ")
            print("Full Name:",full_name)
            print("Company:",company_name)
        end_time = time.time()
        run_time = end_time - start_time
        return run_time
    
    #query2: prints the name and number of employees for all companies
    def query2(self):
        start_time = time.time()
        for company in self.collection.find({"type": "company"}):
            company_name = company['name']
            number_of_employees = len(company['staff'])
            print(" ")
            print("Company:",company_name)
            print("Number of Employees:",number_of_employees)
        end_time = time.time()
        run_time = end_time - start_time
        return run_time
        
    #query3: update the age to be 30 for all persons whose date of birth is before 1988-01-01
    def query3(self):
        start_time = time.time()
        for person in self.collection.find({"type": "person"}):
            dob = person['dateOfBirth']
            if dob < pd.to_datetime('1988-01-01'):
                print(" ")
                print(" Pre - Age Change:",person['age'])
                result = self.collection.update_one(
                            {'_id': person['_id']},
                            {'$set': {'age': 30}}
                        )
                person_ = self.collection.find_one({'_id':person['_id']})
                print("Post - Age Change:",person_['age'])
        end_time = time.time()
        run_time = end_time - start_time
        return run_time

    #query4: update the company name to include the word "Company"    
    def query4(self):
        start_time = time.time()
        for company in self.collection.find({"type": "company"}):
            company_name = company['name']
            print(" ")
            print(" Pre - Name Change:",company_name)
            result = self.collection.update_one(
                        {'_id': company['_id']},
                        {'$set': {'name': "Company "+company_name}}
                    )
            company_ = self.collection.find_one({'_id':company['_id']})
            print("Post - Name Change:",company_['name'])
        end_time = time.time()
        run_time = end_time - start_time
        return run_time


### **Model2**: One document for “Person” with “Company” as embedded document.

In [12]:
class Model2:
    #initialize the collection to hold one document with 'persons' and their respective firms
    #should be the slowest one, I guess.
    def __init__(self, host='127.0.0.1', port=27017, dbname='test'):
        self.client = MongoClient(host, port)
        self.db = self.client[dbname]
        self.db.drop_collection("model2")
        self.collection = self.db.create_collection('model2')

    #create a function that generates data and stores it in the collection
    def data_generator(self, n_company, n_person):
        # create sample data
        fake = Faker(['en_US'])

        # create sample company data
        companies = []
        for c in range(n_company):
            company = {
                'type': 'company',
                '_id': ObjectId(),  # Generate a unique ID for each company
                'domain': fake.domain_word(),
                'email': fake.ascii_company_email(),
                'name': fake.company(),
                'url': fake.uri(),
                'vatNumber': fake.nic_handles()
            }
            companies.append(company)

        # create sample staff data
        persons = []
        for p in range(n_person):
            today = datetime.date.today()
            dob = pd.to_datetime(fake.date_of_birth(minimum_age=18, maximum_age=67))
            age = today.year - dob.year - ((today.month, today.day) < (dob.month, dob.day))
            company = random.choice(companies)  # Choose a random company
            person = {
                'type': 'person',
                '_id': ObjectId(),  # Generate a unique ID for each person
                'age': age,
                'companyEmail': fake.ascii_company_email(),
                'dateOfBirth': dob,
                'email': fake.email(),
                'firstName': fake.first_name(),
                'secondName': fake.last_name(),
                'job': fake.job(),
                'worksIn': company  # Embed the company information within the person document
            }
            persons.append(person)

        # Insert all person documents into the collection
        self.collection.insert_many(persons)


     #query1: prints full name and company for all persons
    def query1(self):
        start_time = time.time()
        for person in self.collection.find({"type": "person"}):
            full_name = person['firstName'] + ' ' + person['secondName']
            company_name = person['worksIn']['name']
            print(" ")
            print("Full Name:", full_name)
            print("Company:", company_name)
        end_time = time.time()
        run_time = end_time - start_time
        return run_time

#query2: prints the name and number of employees for all companies
    def query2(self):
        start_time = time.time()
        companies = {}
        for person in self.collection.find({"type": "person"}):
            company_name = person['worksIn']['name']
            if company_name not in companies.keys():
                companies[company_name] = 1
            else:
                companies[company_name] += 1
        for company, num in companies.items():
            print(" ")
            print('Company:', company, '\n Num of Employees:', num) 
        end_time = time.time()
        run_time = end_time - start_time
        return run_time

    #query3: update the age to be 30 for all persons whose date of birth is before 1988-01-01
    def query3(self):
            start_time = time.time()
            for person in self.collection.find({"type": "person"}):
                dob = person['dateOfBirth']
                if dob < pd.to_datetime('1988-01-01'):
                    print(" ")
                    print(" Pre - Age Change:",person['age'])
                    result = self.collection.update_one(
                                {'_id': person['_id']},
                                {'$set': {'age': 30}}
                            )
                    person_ = self.collection.find_one({'_id':person['_id']})
                    print("Post - Age Change:",person_['age'])
            end_time = time.time()
            run_time = end_time - start_time
            return run_time

    #query4: update the company name to include the word "Company"    
    def query4(self):
        start_time = time.time()
        for person in self.collection.find({"type": "person"}):
            company_name = person['worksIn']['name']
            print(" ")
            print("Pre - Name Change:", company_name)
            
            # Update the company name within the worksIn field of the person document
            result = self.collection.update_one(
                {'_id': person['_id']},
                {'$set': {'worksIn.name': "Company " + company_name}}
            )

            # Fetch and print the updated company name
            updated_person = self.collection.find_one({'_id': person['_id']})
            updated_company_name = updated_person['worksIn']['name']
            print("Post - Name Change:", updated_company_name)

        end_time = time.time()
        run_time = end_time - start_time
        return run_time



### **Model3**: One document for “Company” with “Person” as embedded documents.

In [ ]:
#Model3


## <a id='part2'>Part 2: Query Execution</a>
- **Objective**: Execute four queries and log run time for each model.
- **Tasks:**
  - Query1: For each person, retrieve their full name and their company’s name.
  - Query2: For each company, retrieve its name and the number of employees.
  - Query3: For each person born before 1988, update their age to “30”.
  - Query4: For each company, update its name to include the word “Company”.

In [13]:
#Initialize models and data generation

#Model1
model1 = Model1()
model1.data_generator(NUMBER_OF_COMPANIES, NUMBER_OF_EMPLOYEES)

#model2
model2 = Model2()
model2.data_generator(NUMBER_OF_COMPANIES,NUMBER_OF_EMPLOYEES)
#model3


### **Query1**: For each person, retrieve their full name and their company’s name.


In [5]:
#Model1 - Query1
print("Model1 - Query1==========\n")
q1_m1_run_time = model1.query1()

#Model2 - Query1
print("Model2 - Query1==========\n")
q1_m2_run_time = model2.query1()

#Model3 - Query1
print("Model3 - Query1==========")
#


Model1 - Query1==========

 
Full Name: Veronica Wood
Company: Valentine-Miller
 
Full Name: Tammy Oneill
Company: Valentine-Miller
 
Full Name: Deborah Whitney
Company: Valentine-Miller
 
Full Name: Paul Rodriguez
Company: Valentine-Miller
 
Full Name: Matthew Palmer
Company: Cantrell, Fuller and Santos
 
Full Name: Ashley Brooks
Company: Cantrell, Fuller and Santos
 
Full Name: Heather Allen
Company: Cantrell, Fuller and Santos
 
Full Name: Brenda Sanchez
Company: Cantrell, Fuller and Santos
Model2 - Query1==========

 
Full Name: Devin Meyers
Company: Crawford-Matthews
 
Full Name: Lisa Hardy
Company: Crawford-Matthews
 
Full Name: Rachel Rowe
Company: Crawford-Matthews
 
Full Name: Timothy Browning
Company: Crawford-Matthews
 
Full Name: Scott Carr
Company: Smith, Lucero and Jones
 
Full Name: Thomas Henderson
Company: Smith, Lucero and Jones
 
Full Name: Karina Thompson
Company: Smith, Lucero and Jones
 
Full Name: Wendy Murray
Company: Smith, Lucero and Jones
Model3 - Query1=====

### **Query2**: For each company, retrieve its name and the number of employees.


In [6]:
#Model1 - Query2
print("Model1 - Query2==========")
q2_m1_run_time = model1.query2()

#Model2 - Query2
print("Model2 - Query2==========")
q2_m2_run_time = model2.query2()

#Model3 - Query3
print("Model3 - Query3==========")
#


Model1 - Query2==========
 
Company: Valentine-Miller
Number of Employees: 4
 
Company: Cantrell, Fuller and Santos
Number of Employees: 4
Model2 - Query2==========
 
Company: Crawford-Matthews 
 Num of Employees: 4
 
Company: Smith, Lucero and Jones 
 Num of Employees: 4
Model3 - Query3==========


### **Query3**: For each person born before 1988, update their age to “30”.

In [14]:
#Model1 - Query3
print("Model1 - Query3==========")
q3_m1_run_time = model1.query3()

#Model2 - Query3
print("Model2 - Query3==========")
q3_m2_run_time = model2.query3()

#Model3 - Query3
print("Model3 - Query3==========")
#


Model1 - Query3==========
 
 Pre - Age Change: 36
Post - Age Change: 30
 
 Pre - Age Change: 47
Post - Age Change: 30
 
 Pre - Age Change: 45
Post - Age Change: 30
 
 Pre - Age Change: 62
Post - Age Change: 30
 
 Pre - Age Change: 60
Post - Age Change: 30
 
 Pre - Age Change: 42
Post - Age Change: 30
Model2 - Query3==========
 
 Pre - Age Change: 38
Post - Age Change: 30
 
 Pre - Age Change: 64
Post - Age Change: 30
 
 Pre - Age Change: 64
Post - Age Change: 30
Model3 - Query3==========


### **Query4**: For each company, update its name to include the word “Company”.

In [21]:
#Model1 - Query4
print("Model1 - Query4==========")
q4_m1_run_time = model1.query4()

#Model2 - Query4
print("Model2 - Query4==========")
q4_m2_run_time = model2.query4()


#Model3 - Query4
print("Model3 - Query4==========")
#


Model1 - Query4==========
 
 Pre - Name Change: Company Company Company Company Company Company Mccormick, Moore and Flores
Post - Name Change: Company Company Company Company Company Company Company Mccormick, Moore and Flores
 
 Pre - Name Change: Company Company Company Company Company Company Morris-Hayes
Post - Name Change: Company Company Company Company Company Company Company Morris-Hayes
Model2 - Query4==========
 
Pre - Name Change: Company Company Company Company Company Company Webster Inc
Post - Name Change: Company Company Company Company Company Company Company Webster Inc
 
Pre - Name Change: Company Company Company Company Company Company Webster Inc
Post - Name Change: Company Company Company Company Company Company Company Webster Inc
 
Pre - Name Change: Company Company Company Company Company Company Webster Inc
Post - Name Change: Company Company Company Company Company Company Company Webster Inc
 
Pre - Name Change: Company Company Company Company Company Compan

### Append run time results into a dataframe to present

In [ ]:
#compile run times for Model1
columns = ['Model','Q1_run_time','Q2_run_time','Q3_run_time','Q4_run_time']
ls_m1_run_times = ['Model 1', q1_m1_run_time, q2_m1_run_time, q3_m1_run_time, q4_m1_run_time]
ls_m2_run_times = ['Model 2', q1_m1_run_time, q2_m1_run_time, q3_m1_run_time, q4_m1_run_time]
#ls_m3_run_times = ['Model 2', q1_m3_run_time, q2_m3_run_time, q3_m3_run_time, q4_m3_run_time]


df_m1 = pd.DataFrame([ls_m1_run_times], columns=columns)
df_m2 = pd.DataFrame([ls_m2_run_times], columns=columns)
#df_m3 = pd.DataFrame([ls_m3_run_times], columns=columns)

# Concatenate the DataFrames to create a single DataFrame
df_combined = pd.concat([df_m1, df_m2], ignore_index=True)


## <a id='part3'>Part 3: Results & Discussion</a>
- **Objective**: Compare run times for query execution across the three models.
- **Tasks:** 
  - Question1: Order queries from best to worst for Q1. Which model performs best? Why?
  - Question2: Order queries from best to worst for Q2. Which model performs best? Why?
  - Question3: Order queries from best to worst for Q3. Which model performs best? Why?
  - Question4: Order queries from best to worst for Q4. Which model performs best? Why?
  - Question5: What are your conclusions about denormalization or normalization of data in MongoDB? In the case of updates, which others better performance?

In [ ]:
#table of all the run times of queries goes here

### **Question1**: Order queries from best to worst for Q1. Which model performs best? Why?

### **Question2**: Order queries from best to worst for Q2. Which model performs best? Why?

### **Question3**: Order queries from best to worst for Q3. Which model performs best? Why?

### **Question4**: Order queries from best to worst for Q4. Which model performs best? Why?

### **Question5**: What are your conclusions about denormalization or normalization of data in MongoDB? In the case of updates, which others better performance?